# 1.读取原始数据

In [1]:
import pandas as pd
hu_data = pd.read_excel(r"C:\Users\12045\Desktop\2020-2022年中国江苏省土地经济调查统计数据集\原始数据\户数据（原始数据）.xlsx")
ground_data = pd.read_excel(r"C:\Users\12045\Desktop\2020-2022年中国江苏省土地经济调查统计数据集\原始数据\地块数据（原始数据）.xlsx")
village_data = pd.read_excel(r"C:\Users\12045\Desktop\2020-2022年中国江苏省土地经济调查统计数据集\原始数据\村数据（原始数据）.xlsx")

In [2]:
# 输出列数
num_columns1 = village_data.shape[1]
print("村数据有", num_columns1, "列")
num_columns2 = ground_data.shape[1]
print("地块数据有", num_columns2, "列")
num_columns3 = hu_data.shape[1]
print("户数据有", num_columns3, "列")

村数据有 56 列
地块数据有 206 列
户数据有 206 列


# 2.处理重复值

In [3]:
print("村数据重复值数:", village_data.duplicated().sum())
print("地块数据重复值数:", ground_data.duplicated().sum())
print("户数据重复值数:", hu_data.duplicated().sum())

村数据重复值数: 0
地块数据重复值数: 0
户数据重复值数: 0


# 3.删除明显无法使用的特征列:

数据集只有江苏省数据，故删除省份列；市县村编码不公开无法对应，故删除市县村编码列；mc_chengbaogd，mx_chengbaogd，ms_chengbaogd三列和前三列重复，故删除。村数据最后11列为常数列，地块和户数据最后19列为常数列，故删除

In [4]:
village_data = village_data.drop(columns=['a', 'b', 'c', 'd', 'e'])
ground_data = ground_data.drop(columns=['a', 'b', 'c', 'd', 'e', 'f', 'mc_chengbaogd', 'mx_chengbaogd', 'ms_chengbaogd'])
hu_data = hu_data.drop(columns=['a', 'b', 'c', 'd', 'e', 'f', 'mc_chengbaogd', 'mx_chengbaogd', 'ms_chengbaogd'])
# 删除最后11列
village_data = village_data.iloc[:, :-11]
# 删除最后19列
ground_data = ground_data.iloc[:, :-19]
hu_data = hu_data.iloc[:, :-19]

# 4.处理异常值

## 4.1先处理完全不符合实际的数据

### 4.1.1将村数据承包地面积为0的数据替换为空值，已流转承包地面积大于承包地面积的数据替换为承包地面积

In [5]:
# 将村数据承包地面积为0的数据替换为空值
import numpy as np
zero_count = (village_data['chengbaodiarea'] == 0).sum()
print("村数据的“承包地面积”列中的0值的数量为:", zero_count)
# 将chengbaodiarea列中的0替换为空值
village_data['chengbaodiarea'].replace(0, np.nan, inplace=True)

# 已流转承包地面积大于承包地面积的数据替换为承包地面积
count=0
for index, row in village_data.iterrows():
    # 如果这一行的两个数值都不是空值，则进行比较和替换操作
    if pd.notnull(row['liuzhuandiarea']) and pd.notnull(row['chengbaodiarea']):
        if row['liuzhuandiarea'] > row['chengbaodiarea']:
            # 计算比较和替换的数量
            count += 1
            # 将 liuzhuandiarea 列中大于对应的 chengbaodiarea 列的值替换为 chengbaodiarea 列的值
            village_data.at[index, 'liuzhuandiarea'] = row['chengbaodiarea']

print("“已流转承包地” 列中大于对应的 “承包地面积” 列的数值的数量为:", count)

print("替换完成")

村数据的“承包地面积”列中的0值的数量为: 3
“已流转承包地” 列中大于对应的 “承包地面积” 列的数值的数量为: 13
替换完成


### 4.1.2 把地块数据和户数据中“耕地承包的（分到的）总面积（亩）”列和“耕地承包的（分到的）地块数量（块/片）”列中的0替换成空，把所有999（代表不知道）替换成空

In [6]:
# 替换两列中的0为空
zero_count1 = (ground_data['chengbaogd'] == 0).sum()
zero_count2 = (ground_data['chengbaogdks'] == 0).sum()
zero_count3 = (hu_data['chengbaogd'] == 0).sum()
zero_count4 = (hu_data['chengbaogdks'] == 0).sum()
print("地块数据的“耕地承包的（分到的）总面积（亩）”列中的0值的数量为:", zero_count1)
ground_data['chengbaogd'].replace(0, np.nan, inplace=True)
print("地块数据的“耕地承包的（分到的）地块数量（块/片）”列中的0值的数量为:", zero_count2)
ground_data['chengbaogdks'].replace(0, np.nan, inplace=True)
print("户数据的“耕地承包的（分到的）总面积（亩）”列中的0值的数量为:", zero_count3)
hu_data['chengbaogd'].replace(0, np.nan, inplace=True)
print("户数据的“耕地承包的（分到的）地块数量（块/片）”列中的0值的数量为:", zero_count4)
hu_data['chengbaogdks'].replace(0, np.nan, inplace=True)

# 替换999为空
replace_count1 = (ground_data == 999).sum().sum()
ground_data.replace(999, 0, inplace=True)
print("地块数据中999的个数:", replace_count1)
replace_count2 = (hu_data == 999).sum().sum()
hu_data.replace(999, 0, inplace=True)
print("户数据中999的个数:", replace_count2)

print("替换完成")

地块数据的“耕地承包的（分到的）总面积（亩）”列中的0值的数量为: 0
地块数据的“耕地承包的（分到的）地块数量（块/片）”列中的0值的数量为: 0
户数据的“耕地承包的（分到的）总面积（亩）”列中的0值的数量为: 0
户数据的“耕地承包的（分到的）地块数量（块/片）”列中的0值的数量为: 5
地块数据中999的个数: 82
户数据中999的个数: 812
替换完成


## 4.2处理地块数据和户数据中的分类数据中的非编码部分，将其转换为对应编码，用正则表达式匹配第一个数值，把原数据替换为匹配到的数值

In [7]:
# 输出地块数据分类数据中不止有编码的特征列
print("地块数据分类数据中不止有编码的特征列:")
# 输出非空且不全是数字的数据以及对应的列名
for col in ground_data.columns:
    non_numeric_values = ground_data[col].dropna().loc[ground_data[col].apply(lambda x: isinstance(x, str) and not x.isdigit())]
    if not non_numeric_values.empty:
        print("列名:", col)
        print(non_numeric_values.head(20))
        print("-------------------")


地块数据分类数据中不止有编码的特征列:
列名: cbdpodu
838    4丘陵
861    4丘陵
Name: cbdpodu, dtype: object
-------------------
列名: zrdpodu
97     4池塘
838    4丘陵
858    4水塘
Name: zrdpodu, dtype: object
-------------------
列名: cbdturang
514    4沙土黏土混合
608      4沙土黑土
792        4混合
Name: cbdturang, dtype: object
-------------------
列名: zrdturang
514    4沙土黏土混合
608      4沙土黑土
614        4白土
858        4水塘
Name: zrdturang, dtype: object
-------------------
列名: cbdyongtu201912
7         8a
516      1,2
838    12,10
Name: cbdyongtu201912, dtype: object
-------------------
列名: zrdyongtu201912
9           3,4
10          1,2
45          18竹
516         1,2
709         1,2
815    2,14,18鹅
Name: zrdyongtu201912, dtype: object
-------------------
列名: cbdyongtu202008
7         8a
516      1,2
585     1,14
657       8a
838    12,10
Name: cbdyongtu202008, dtype: object
-------------------
列名: zrdyongtu202008
45          18竹
516         1,2
585        1,14
657          8a
815    1,14,18鹅
Name: zrdyongtu202008, dtype: object


In [8]:
# 转换为只有编码
import re

# 定义一个函数来转换非空且不全是数字的数据
def convert_to_numeric(value):
    # 匹配数字和逗号
    pattern = re.compile(r'\d+')
    # 查找匹配的数字
    match = pattern.search(value)
    if match:
        return int(match.group())  # 返回第一个匹配到的数字
    else:
        return None  # 如果找不到数字，则返回 None

# 遍历每一列
for col in ground_data.columns:
    non_numeric_indices = ground_data[col].dropna().loc[ground_data[col].apply(lambda x: isinstance(x, str) and not x.isdigit())].index
    # 如果非数字值不为空
    if not non_numeric_indices.empty:
        # 对于每个非数字值，将其转换为数字
        for idx in non_numeric_indices:
            ground_data.at[idx, col] = convert_to_numeric(ground_data.at[idx, col])
print("转换完成")

转换完成


In [9]:
# 输出户数据分类数据中不止有编码的特征列
print("户数据分类数据中不止有编码的特征列:")
# 输出非空且不全是数字的数据
for col in hu_data.columns:
    non_numeric_values = hu_data[col].dropna().loc[hu_data[col].apply(lambda x: isinstance(x, str) and not x.isdigit())]
    if not non_numeric_values.empty:
        print("列名:", col)
        print(non_numeric_values.head(20))
        print("-------------------")


户数据分类数据中不止有编码的特征列:
列名: cbdpodu
434          .
4749       4丘陵
4819    4平地和坡地
4837       4丘陵
4950       4丘陵
5365       4水面
5518       4水塘
Name: cbdpodu, dtype: object
-------------------
列名: zrdpodu
513     4池塘
4837    4丘陵
4929    4水塘
Name: zrdpodu, dtype: object
-------------------
列名: cbdturang
36          4水田
334         4水田
526         4水田
531         4水田
2798    4:1和2混合
2839      412都有
2860    4沙土黏土混合
3130        4混合
3153      4黑泥黄泥
3172      4沙土黑土
3233        4粪土
3879       4沙壤土
4593        4混合
4738        4泥土
5014       4混合土
5231       4不清楚
5334        4水田
5362        4红土
5427        4红土
5664        4红土
Name: cbdturang, dtype: object
-------------------
列名: zrdturang
2860    4沙土黏土混合
3172      4沙土黑土
3182        4白土
4929        4水塘
Name: zrdturang, dtype: object
-------------------
列名: cbdyongtu201912
19       8b
57       8a
74       8a
85       8a
93      2,3
99       8a
102      8b
132     1,6
153      8b
163    18草莓
165    18草莓
170      8b
183      8b
221    18转出
238      8a
288 

In [10]:
# 转换为只有编码
import re

# 定义一个函数来转换非空且不全是数字的数据
def convert_to_numeric(value):
    # 匹配数字和逗号
    pattern = re.compile(r'\d+')
    # 查找匹配的数字
    match = pattern.search(value)
    if match:
        return int(match.group())  # 返回第一个匹配到的数字
    else:
        return None  # 如果找不到数字，则返回 None

# 遍历每一列
for col in hu_data.columns:
    non_numeric_indices = hu_data[col].dropna().loc[hu_data[col].apply(lambda x: isinstance(x, str) and not x.isdigit())].index
    # 如果非数字值不为空
    if not non_numeric_indices.empty:
        # 对于每个非数字值，将其转换为数字
        for idx in non_numeric_indices:
            hu_data.at[idx, col] = convert_to_numeric(hu_data.at[idx, col])
            
print("转换完成")

转换完成


## 4.3处理村数据异常值：把连续性数据中z-score>3的数据当作异常值，并转换为空值

In [11]:
# 处理村数据异常值
import numpy as np
from scipy.stats import zscore

# 初始化一个空列表，用于存储异常值的索引
outlier_indices = []
# 初始化一个空字典，用于存储每一列中的异常值个数
outlier_counts = {}
# 遍历每一列
for col in village_data.columns:
    # 判断是否是常数列或者是quequanyear列
    if village_data[col].nunique() > 1 and col != 'quequanyear' and col != 'year':
        # 计算 z-score
        z_scores = zscore(village_data[col])
        # 找到 z-score 绝对值大于 3 的索引
        col_indices = np.where(np.abs(z_scores) > 3)[0]
        # 统计异常值个数
        count = len(col_indices)
        if count > 0:
            outlier_counts[col] = count
            # 将异常值的索引添加到列表中
            outlier_indices.extend(col_indices)

# 去除重复的索引
outlier_indices = list(set(outlier_indices))

# 输出异常值个数
for col, count in outlier_counts.items():
    print(f"列 {col} 中的异常值个数: {count}")

# 将异常值替换为 nan
for idx in outlier_indices:
    for col in village_data.columns:
        if village_data[col].nunique() > 1 and col != 'quequanyear' and col != 'year':
            if np.abs(zscore(village_data[col])[idx]) > 3:
                village_data.at[idx, col] = np.nan
print("替换成功")

列 liuzhuandiarea 中的异常值个数: 2
列 zhuti500 中的异常值个数: 2
列 zhuti200 中的异常值个数: 4
列 zhuti100 中的异常值个数: 3
列 zhuti50 中的异常值个数: 1
列 zhutiwaicun 中的异常值个数: 3
替换成功


## 4.4处理地块数据和户数据异常值：由于地块数据和户数据样本真实情况偏离程度较大，所以把连续性数据中z-score>5.5的数据才当作异常值

In [12]:
# 处理地块数据
import numpy as np
from scipy.stats import zscore
# 初始化一个空列表，用于存储异常值的索引
outlier_indices = []
# 初始化一个空字典，用于存储每一列中的异常值个数
outlier_counts = {}
# 分类数据不遍历
no_selected_col = ["cbdfangwei", "zrdfangwei", "zjxingshi", "c214b", "cbdpodu", "zrdpodu", "cbdgaosulu",
                   "zrdgaosulu", "cbdturang", "zrdturang", "cbdguangai", "zrdguangai", "cbdfeili", "zrdfeili",
                   "cbdyongtu201912", "zrdyongtu201912", "cbdyongtu202008", "zrdyongtu202008", "cbdwrxiufu",
                   "zrdwrxiufu", "shifouzc", "cbdjinqin", "zrdjinqin", "cbdbutie", "zrdbutie", "cbdxietiao",
                   "zrdxietiao", "cbdhetong", "zrdhetong", "cbdjypt", "zrdjypt", "cbdjyptreason1",
                   "zrdjyptreason1", "cbdjyptreason2", "zrdjyptreason2", "cbdjiangqixian", "zrdjiangqixian",
                   "d301a", "d301b", "d311a", "d311b", "d312a", "d312b", "d313a", "d313b", "d314a", "d314b",
                   "d315a", "d315b", "d316a", "d316b", "d317a", "d317b", "d319a", "d319b", "d320a", "d320b",
                   "d321a", "d321b", "d322a", "d322b", "d323a", "d323b", "d324a", "d324b", "d325a", "d325b",
                   "d328a", "d328b", "d329a", "d329b", "d331a", "d331b", "d335a", "d335b", "d336a", "d336b",
                   "d337a", "d337b", "d338a", "d338b", "d339a", "d339b", "d340a", "d340b", "d341a", "d341b"
                   "cbdpodu","zrdpodu","cbdturang","zrdturang","cbdyongtu201912","zrdyongtu201912",
                   "cbdyongtu202008","zrdyongtu202008","cbdjiangqixian","cbdqixian","zrdqixian","d301a","d301b","d336a","d336b","hid"]

# 遍历每一列
for col in ground_data.columns:
    if  col not in no_selected_col:
        # 计算 z-score
        z_scores = zscore(ground_data[col])
        # 找到 z-score 绝对值大于 5.5 的索引
        col_indices = np.where(np.abs(z_scores) > 5.5)[0]
        # 统计异常值个数
        count = len(col_indices)
        if count > 0:
            outlier_counts[col] = count
            # 将异常值的索引添加到列表中
            outlier_indices.extend(col_indices)

# 去除重复的索引
outlier_indices = list(set(outlier_indices))

# 输出异常值个数
for col, count in outlier_counts.items():
    print(f"列 {col} 中的异常值个数: {count}")

# 将异常值替换为 NaN
for idx in outlier_indices:
    for col in ground_data.columns:
        if ground_data[col].nunique() > 1 and col not in no_selected_col:
            if np.abs(zscore(ground_data[col])[idx]) > 5.5:
                ground_data.at[idx, col] = np.nan
print("替换成功")

列 chengbaogd 中的异常值个数: 5
列 zcgdarea 中的异常值个数: 2
列 zcqiyearea 中的异常值个数: 8
列 zchezuoshearea 中的异常值个数: 3
列 zccunjitiarea 中的异常值个数: 4
列 zcnonghuarea 中的异常值个数: 4
列 zcareahetong 中的异常值个数: 4
列 zcareaqixian 中的异常值个数: 3
列 jitizcarea 中的异常值个数: 3
列 jitizcareacgb 中的异常值个数: 3
列 zrdarea 中的异常值个数: 7
列 cychengbaogd 中的异常值个数: 3
替换成功


In [13]:
# 处理户数据
import numpy as np
from scipy.stats import zscore
# 初始化一个空列表，用于存储异常值的索引
outlier_indices = []
# 初始化一个空字典，用于存储每一列中的异常值个数
outlier_counts = {}
# 分类数据不遍历
no_selected_col = ["cbdfangwei", "zrdfangwei", "zjxingshi", "c214b", "cbdpodu", "zrdpodu", "cbdgaosulu",
                   "zrdgaosulu", "cbdturang", "zrdturang", "cbdguangai", "zrdguangai", "cbdfeili", "zrdfeili",
                   "cbdyongtu201912", "zrdyongtu201912", "cbdyongtu202008", "zrdyongtu202008", "cbdwrxiufu",
                   "zrdwrxiufu", "shifouzc", "cbdjinqin", "zrdjinqin", "cbdbutie", "zrdbutie", "cbdxietiao",
                   "zrdxietiao", "cbdhetong", "zrdhetong", "cbdjypt", "zrdjypt", "cbdjyptreason1",
                   "zrdjyptreason1", "cbdjyptreason2", "zrdjyptreason2", "cbdjiangqixian", "zrdjiangqixian",
                   "d301a", "d301b", "d311a", "d311b", "d312a", "d312b", "d313a", "d313b", "d314a", "d314b",
                   "d315a", "d315b", "d316a", "d316b", "d317a", "d317b", "d319a", "d319b", "d320a", "d320b",
                   "d321a", "d321b", "d322a", "d322b", "d323a", "d323b", "d324a", "d324b", "d325a", "d325b",
                   "d328a", "d328b", "d329a", "d329b", "d331a", "d331b", "d335a", "d335b", "d336a", "d336b",
                   "d337a", "d337b", "d338a", "d338b", "d339a", "d339b", "d340a", "d340b", "d341a", "d341b"
                   "cbdpodu","zrdpodu","cbdturang","zrdturang","cbdyongtu201912","zrdyongtu201912",
                   "cbdyongtu202008","zrdyongtu202008","cbdjiangqixian","cbdqixian","zrdqixian","d301a","d301b","d336a","d336b",
                   "cbdpodu","zrdpodu","cbdturang","zrdturang","cbdyongtu201912","zrdyongtu201912","cbdyongtu202008",
                   "zrdyongtu202008","cbdjiangqixian","cbdqixian","zrdqixian","d301a","d301b","d331a","d336a","d339a","hid"]

# 遍历每一列
for col in hu_data.columns:
    if  col not in no_selected_col:
        # 计算 z-score
        z_scores = zscore(hu_data[col])
        # 找到 z-score 绝对值大于 5.5 的索引
        col_indices = np.where(np.abs(z_scores) > 5.5)[0]
        # 统计异常值个数
        count = len(col_indices)
        if count > 0:
            outlier_counts[col] = count
            # 将异常值的索引添加到列表中
            outlier_indices.extend(col_indices)

# 去除重复的索引
outlier_indices = list(set(outlier_indices))

# 输出异常值个数
for col, count in outlier_counts.items():
    print(f"列 {col} 中的异常值个数: {count}")

# 将异常值替换为 nan
for idx in outlier_indices:
    for col in hu_data.columns:
        if hu_data[col].nunique() > 1 and col not in no_selected_col:
            if np.abs(zscore(hu_data[col])[idx]) > 5.5:
                hu_data.at[idx, col] = np.nan
print("替换成功")

列 chengbaogd 中的异常值个数: 19
列 zcgdarea 中的异常值个数: 11
列 zcqiyearea 中的异常值个数: 48
列 zchezuoshearea 中的异常值个数: 34
列 zccunjitiarea 中的异常值个数: 24
列 zcnonghuarea 中的异常值个数: 11
列 zcareahetong 中的异常值个数: 12
列 zcareaqixian 中的异常值个数: 7
列 jitizcarea 中的异常值个数: 4
列 jitizcareacgb 中的异常值个数: 12
列 cychengbaogd 中的异常值个数: 24
替换成功


# 5.处理缺失值：看下一文件

# 6.输出预处理后文件

In [14]:
# 输出预处理后数据
village_data.to_csv(r"C:\Users\12045\Desktop\村数据(预处理中).csv", index=False)
ground_data.to_csv(r"C:\Users\12045\Desktop\地块数据(预处理中).csv", index=False)
hu_data.to_csv(r"C:\Users\12045\Desktop\户数据(预处理中).csv", index=False)